# Behavioural Cloning by Atif

Import all libraries used in the model 

In [ ]:
import csv
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Flatten,Dense,Lambda
from keras.layers import Convolution2D
from keras.layers import Cropping2D
from keras import backend as K

Read Training data set(s). 1st read given training data.

Then additional training sets created with 10Hz Simulator using keyboard input

In [ ]:

K.set_image_dim_ordering('tf')
train_data=[]

with open('driving_log.csv') as train_file:
    read = csv.reader(train_file)
    for line in read:
        train_data.append(line)

train_data2=[]
with open('driving_log2.csv') as train_file:
    read = csv.reader(train_file)
    for line in read:
        train_data2.append(line)
        
train_data3=[]
with open('driving_log3.csv') as train_file:
    read = csv.reader(train_file)
    for line in read:
        train_data3.append(line)


Combine the input training sets. 

From training sets, use all 3 camera angle views, 
    1. center_image as is with given steering angle
    2. left_image with extra 0.3 steered right
    3. right_image with extra 0.3 steered left
The optimal steering delta for left/right images was evaluated after trying couple values.  

In [ ]:
image_set = []
train_data = train_data[1:] + train_data2 + train_data3
print(len(train_data))
output = []
i = 1
for data in train_data:
    i+=1
    center_image = data[0]
    left_image = data[1].strip()
    right_image=data[2].strip()

    image_set.extend([cv2.imread(center_image),cv2.imread(left_image),cv2.imread(right_image)])
    output.extend([data[3],float(data[3])+0.3,float(data[3])-0.3])   

X_train = np.array(image_set)
y_train = np.array(output)

print(X_train.shape)

The actual model.
As recommended, i used NVidea's Convolutional Neural Network Model, with 5 Conv2D + 3 FullyConnected layers. 
![NVidea.png](attachment:NVidea.png)
[NVidea]: ./NVidea.png "Model Visualization"

As loss did not reduce after few EPOCHs, restricted the model to just 4 EPOCHs. 

In [1]:
model=Sequential()
model.add(Lambda(lambda x: x/255.0-0.5,input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Convolution2D(64,3,3,activation="relu"))

model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))


model.compile(loss='mse',optimizer='adam')

model.fit(X_train,y_train,validation_split=0.2,shuffle=True,nb_epoch=4)
model.save('Naive_model.h5')



Using TensorFlow backend.


15708
(47124, 160, 320, 3)
Train on 37699 samples, validate on 9425 samples
Epoch 1/4
37699/37699 [==============================] - 95s - loss: 0.0456 - val_loss: 0.1102